In [1]:
import torch
import numpy as np

In [2]:
from torch import nn
import matplotlib.pyplot as plt

In [3]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

## Creating A Case of MultiVariate Regression ##

Suppose, dependent Variable $y$ depends on 2 Independant Variables $X_1$ & $X_2$, 
<br> as <br/>
<br> $$y = 3X_1 + 5X_2 + C$$ <br/>

A Physical example would be

1. $Y$ is flow rate measurement from pipe origining from 2 points. $X_1$ & $X_2$ is known flowrate measurement.
   - $C$ is added to account for the measurement error



In [4]:
wt = torch.tensor([3, 5]) 
bias = torch.tensor([11])

## Creation of Synthetic Data

In [5]:
X_1 = torch.randn(50).unsqueeze(dim=1)

In [6]:
X_2 = torch.randn(50).unsqueeze(dim=1)

In [7]:
X_tot = torch.stack([X_1, X_2], dim=1).squeeze(dim=2)

## Ground Truth

$$ y = 3 X_1 + 5 X_2 + 11

In [8]:
y = (wt * X_tot).sum(dim=1) + bias

In [9]:
train_split = int(0.8 * len(X_tot))

In [10]:
X_train, y_train = X_tot[:train_split], y[:train_split]
X_test, y_test = X_tot[train_split:], y[train_split:]

## Shifting Data to GPU

In [11]:
X_train = X_train.to(device)
X_test = X_test.to(device)

y_train, y_test = y_train.to(device), y_test.to(device)

In [12]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     input_var = 'X1',
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  input_xs_slice_num = 0 if input_var == 'X1' else 1

  # Plot training data in blue
  plt.scatter(train_data[:, input_xs_slice_num], train_labels, c="b", s=4, label="Training data - X1")
  
  # Plot test data in green
  plt.scatter(test_data[:, input_xs_slice_num], test_labels, c="g", s=4, label="Testing data - X1")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data[:, input_xs_slice_num], predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14})

In [13]:
import plotly.graph_objects as go

In [14]:
def plot3d(x_train: torch.tensor, y_train: torch.tensor) -> go.Figure:
    """_summary_

    :param train: _description_
    :type train: torch.tensor

    :param test: _description_
    :type test: torch.tensor

    :return: _description_
    :rtype: go.Figure
    """

    var_1, var_2 = x_train[:, 0] , x_train[:, 1]

    y = y_train

    fig = go.Figure()

    fig = go.Figure(data =[go.Scatter3d(x = var_1,
                                   y = var_2,
                                   z = y_train,
                                   mode ='markers')])
    
    return fig


In [15]:
chk = plot3d(x_train=X_train, y_train=y_train)

TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [15]:
chk.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [77]:
## Creating Linear Regression Model Class

In [16]:
torch.randn(1)

tensor([-0.1067])

In [16]:
class LinearRegressionModel(nn.Module):
    """Linear Regression 

    :param nn: _description_
    :type nn: _type_
    """

    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.weights = nn.Parameter(torch.randn(2, 
                                                dtype=torch.float), 
                                                requires_grad=True)

        self.bias = nn.Parameter(torch.randn(1, 
                                             dtype=torch.float), 
                                             requires_grad=True)
        

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return (self.weights * x).sum(dim=1) + self.bias

In [17]:
torch.manual_seed(42)

In [18]:
model_0 = LinearRegressionModel()

In [19]:
model_0.to(device)

LinearRegressionModel()

In [20]:
next(model_0.parameters()).device

device(type='mps', index=0)

## Inference Model in PyTorch

In [21]:
model_0.eval()
with torch.inference_mode():
    y_preds = model_0(X_test)

In [22]:
y_preds

tensor([ 0.3722,  0.4445, -0.0120,  0.0370,  0.5946, -0.0324,  0.6889,  0.4123,
         0.1748,  0.5201], device='mps:0')

## Specifying Loss Functions & Weights Optimizer

In [23]:
list(model_0.parameters())

[Parameter containing:
 tensor([0.3367, 0.1288], device='mps:0', requires_grad=True),
 Parameter containing:
 tensor([0.2345], device='mps:0', requires_grad=True)]

In [24]:
# Loss Function ##
loss_fn = nn.L1Loss()

# Optimizer #
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

In [25]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367, 0.1288], device='mps:0')),
             ('bias', tensor([0.2345], device='mps:0'))])

## Training Loop 

In [26]:
torch.manual_seed(42)

epochs = 5000

for epoch in range(epochs):

    ## Put Model into Training Mode
    model_0.train()

    ## 1. Forward Pass on Train Data using the forward() method

    y_pred = model_0(X_train)

    # 2. Calculate the loss for an entire Eporch #
    loss = loss_fn(y_pred, y_train)

    # 3. Optimizer Zero Grad. At Each Epoch change in parameters is set to 0
    optimizer.zero_grad()

    # 4. Calculate wts increment via Back Prop
    loss.backward()

    # 5. Step the Optimizer (Perform Gradient Descent)
    optimizer.step() # How Optimizer changes will acculumate 

    ### Testing / Evaluate

    model_0.eval() ## Turns off different settings in model that is not relevent for training
    
    with torch.inference_mode(): ## Turns of Gradient Tracking 
        test_pred = model_0(X_test)

        test_loss = loss_fn(test_pred, y_test)

    if epoch % 1000 == 0:
        print(f" Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}")

/Users/niteshkumarsharma/Desktop/AI/PyTorch/.venv/lib/python3.9/site-packages/torch/autograd/__init__.py:200: UserWarning:

The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)



 Epoch: 0 | Loss: 12.286921501159668 | Test Loss: 11.980545043945312
 Epoch: 1000 | Loss: 2.1239147186279297 | Test Loss: 2.0755507946014404
 Epoch: 2000 | Loss: 0.006426775362342596 | Test Loss: 0.004069996066391468
 Epoch: 3000 | Loss: 0.006426775362342596 | Test Loss: 0.004069996066391468
 Epoch: 4000 | Loss: 0.006426775362342596 | Test Loss: 0.004069996066391468


In [108]:
model_0.eval()
with torch.inference_mode():
    y_preds_new = model_0(X_test)

In [32]:
from colab_ssh import launch_ssh
launch_ssh()

Note: you may need to restart the kernel to use updated packages.


In [109]:
y_test

tensor([15.3136, 13.2880, 15.3493,  8.7775,  1.4544, 17.1985,  9.2931, 17.4443,
         9.2918, 12.2589])

OrderedDict([('weights', tensor([-0.2138])), ('bias', tensor([-1.3780]))])

In [91]:
np.abs(y_preds_new - y_test).mean()

tensor(0.0052)

In [92]:
model_0.state_dict()

OrderedDict([('weights', tensor([3.0023, 4.9957])),
             ('bias', tensor([10.9969]))])

In [93]:
import numpy as np

In [116]:
list(model_0.parameters())

[Parameter containing:
 tensor([3.0023, 4.9957], requires_grad=True),
 Parameter containing:
 tensor([10.9969], requires_grad=True)]

## Saving & Loading The Model

In [97]:
## Creating Folder to Write PyTorch Model ##

from pathlib import Path

## 1. Create Models Directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

In [98]:
## 2. Create Model Save Path
MODEL_NAME = "01_pytorch_workflow_MultiVarModel_0.pth"
MODEL_SAVE_PATH = MODEL_PATH /MODEL_NAME

In [99]:
## 3. Save the Model State Dict
torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH)

## Loading Saved PyTorch Model

> Model's `state_dict()` contains the learned paramters, so instead of saving entire PyTorch Model, we create a instance of model and pass the `state_dict()`

`Disadvantage of saving Whole Model is that the serialized data is bound to the specific classes and the exact directory structure when the model is saved.. Making likely that the Code Will Break if refractor is done` `



In [118]:
loaded_reg_mod = LinearRegressionModel()

In [101]:
## Loading State Dict of the Saved Model 
## Essentially Updating the state dict (Which are Random) of Model with Learned State Dict

In [119]:
learned_params_dict = torch.load(f=MODEL_SAVE_PATH)

In [120]:
learned_params_dict

OrderedDict([('weights', tensor([3.0023, 4.9957])),
             ('bias', tensor([10.9969]))])

In [121]:
loaded_reg_mod.load_state_dict(learned_params_dict)

<All keys matched successfully>

## Getting Prediction Out of The Model

In [122]:
# 1. Set Model into eval Mode. This Disables Drop Out , BatchNorm Layers
loaded_reg_mod.eval()

with torch.inference_mode():
    loaded_model_preds = loaded_reg_mod(X_test)


In [123]:
loaded_model_preds 

tensor([15.3085, 13.2820, 15.3403,  8.7725,  1.4586, 17.1927,  9.2918, 17.4378,
         9.2929, 12.2516])

In [127]:
y_preds_new == loaded_model_preds

tensor([True, True, True, True, True, True, True, True, True, True])

In [129]:
X_tot.shape

torch.Size([50, 2])

In [130]:
chk = nn.Linear(2, 1)

In [132]:
chk.state_dict()

OrderedDict([('weight', tensor([[0.5224, 0.0958]])),
             ('bias', tensor([0.3410]))])